<a href="https://colab.research.google.com/github/NourEldin-Osama/T5_Fine-tuning_Text-simplification/blob/main/Notebooks/fine_tuning_t5_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets sacrebleu sacremoses huggingface_hub
!pip install wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 21.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=daf48c6918d2de6ac4356cfbe89560f1d36da99510e453fae2ed76e1b70564ee
  Stored in directory: /root/.cache/pip/wheels/5b/e0/77/05245143a5b31f65af6a21f7afd3219e9fa4896f918af45677
Successfully built sacremoses
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.13.10
    Uninstalling wandb-0.13.10:
      Successfully uninstalled wandb-0.13.10


In [ ]:
import numpy as np
from datasets import load_dataset, load_metric
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, 
                          DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments, Seq2SeqTrainer)
import logging


logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere
# logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
raw_datasets = load_dataset("wiki_auto", "manual")
raw_datasets

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/373801 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/73249 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/118074 [00:00<?, ? examples/s]

Dataset wiki_auto downloaded and prepared to /root/.cache/huggingface/datasets/wiki_auto/manual/1.0.0/eeac705719dc9aa2ff180571dfed6c6649588ccdfde8d45a47d2e47e5c5b93af. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['alignment_label', 'normal_sentence_id', 'simple_sentence_id', 'normal_sentence', 'simple_sentence', 'gleu_score'],
        num_rows: 373801
    })
    dev: Dataset({
        features: ['alignment_label', 'normal_sentence_id', 'simple_sentence_id', 'normal_sentence', 'simple_sentence', 'gleu_score'],
        num_rows: 73249
    })
    test: Dataset({
        features: ['alignment_label', 'normal_sentence_id', 'simple_sentence_id', 'normal_sentence', 'simple_sentence', 'gleu_score'],
        num_rows: 118074
    })
})

In [ ]:
raw_datasets["train"][0]

{'alignment_label': 1,
 'normal_sentence_id': '0_66252-1-0-0',
 'simple_sentence_id': '0_66252-0-0-0',
 'normal_sentence': 'The Local Government Act 1985 is an Act of Parliament in the United Kingdom.',
 'simple_sentence': 'The Local Government Act 1985 was an Act of Parliament in the United Kingdom.',
 'gleu_score': 0.800000011920929}

In [ ]:
max_length = 512
prefix = "simplify: "

def preproces_function(elements):
    inputs = [prefix + doc for doc in elements["normal_sentence"]]
    outputs = elements["simple_sentence"]
    
    model_inputs = tokenizer(inputs, text_target=outputs, max_length=max_length, truncation=True)

    return model_inputs

In [ ]:
model_checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


tokenized_datasets = raw_datasets.map(preproces_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

model_name = model_checkpoint.split("/")[-1]

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


  0%|          | 0/374 [00:00<?, ?ba/s]

  0%|          | 0/74 [00:00<?, ?ba/s]

  0%|          | 0/119 [00:00<?, ?ba/s]

In [ ]:
# num_train_epochs = 1
train_batch_size=16
eval_batch_size=16
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-text-simplification",
    save_total_limit = 1,
    load_best_model_at_end = True,
    save_strategy = "epoch",
    evaluation_strategy="epoch",
    predict_with_generate=True,
    fp16=True,
    include_inputs_for_metrics = True,
#     num_train_epochs=num_train_epochs,
    overwrite_output_dir = True,
    push_to_hub=True,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
#     report_to="none",
    )

In [ ]:
metric = load_metric("sari")

def compute_metrics(eval_pred):
    predictions, labels, inputs = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    decoded_inputs = [[input.strip()] for input in decoded_inputs]

    result = metric.compute(sources=decoded_inputs, predictions=decoded_preds, references=decoded_labels)
    print(result)
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/NourEldin-Osama/t5-small-finetuned-text-simplification into local empty directory.


In [ ]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Sari
1,1.656700,4.510199,58.185275
2,3.765500,4.911852,57.233430
3,3.749800,4.911852,57.233430


{'sari': 58.18527507670439}
{'sari': 57.23342972855014}
{'sari': 57.23342972855014}


TrainOutput(global_step=70089, training_loss=2.9121790504681435, metrics={'train_runtime': 10973.1902, 'train_samples_per_second': 102.195, 'train_steps_per_second': 6.387, 'total_flos': 2.4335943078445056e+16, 'train_loss': 2.9121790504681435, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub("End of Training")

In [ ]:
trainer.save_model("t5-finetuned-text-simplification")

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Mar23_17-55-26_ce32a332d75e/events.out.tfevents.1679594139.ce32a332d75e.23.0:   0%|          …

To https://huggingface.co/NourEldin-Osama/t5-small-finetuned-text-simplification
   04e3806..7f71071  main -> main

To https://huggingface.co/NourEldin-Osama/t5-small-finetuned-text-simplification
   7f71071..7755f85  main -> main

In [ ]:
# After training, access the path of the best checkpoint like this
best_checkpoint_path = trainer.state.best_model_checkpoint
best_checkpoint_path

't5-small-finetuned-text-simplification/checkpoint-23363'

In [ ]:
# best_model = AutoModelForSeq2SeqLM.from_pretrained(best_checkpoint_path)
# best_model.push_to_hub("t5-small-finetuned-text-simplification")